## **Clustering Challenge**

Clustering is an *unsupervised* machine learning technique in which you train a model to group similar entities into clusters based on their features.

In this exercise, you must separate a dataset consisting of three numeric features (**A**, **B**, and **C**) into clusters.

Let's begin by importing the data.


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Read the csv file into a tibble
data <- read_csv(file = "https://raw.githubusercontent.com/MicrosoftDocs/ml-basics/master/challenges/data/clusters.csv", show_col_types = FALSE)

# Print the first 10 rows of the data
data %>% 
  slice_head(n = 10)


Your challenge is to identify the number of discrete clusters present in the data, and create a clustering model that separates the data into that number of clusters. You should also visualize the clusters to evaluate the level of separation achieved by your model.

Add markdown and code cells as required to create your solution.

### **Use PCA to create a 2D version of the features for visualization**

*`Principal Component Analysis`* (PCA) is a dimension reduction method that aims at reducing the feature space, such that, most of the information or variability in the data set can be explained using fewer uncorrelated features.

Let's see this in action by creating a specification of a `recipe` that will estimate the *principal components* based on our three variables. We'll then `prep` and`bake` the recipe to apply the computations.

> PCA works well when the variables are normalized (`centered` and `scaled`)


In [ ]:
# Load the core tidymodels packages and make them available in your current R session
library(tidymodels)

# Specify a recipe for pca
pca_rec <- recipe(~ ., data = data) %>% 
  step_normalize(all_predictors()) %>% 
  step_pca(all_predictors(), num_comp = 2, id = "pca")

# Print out recipe
pca_rec


Compared to supervised learning techniques, we have no `outcome` variable in this recipe.

By calling `prep()` which estimates the statistics required by PCA and applying them to our data using `bake(new_data = NULL)`, we can get the fitted PC transformation of our features.


In [ ]:
# Estimate required statistcs 
pca_estimates <- prep(pca_rec)

# Return preprocessed data using bake
features_2d <- pca_estimates %>% 
  bake(new_data = NULL)

# Print baked data set
features_2d %>% 
  slice_head(n = 5)


🤩 These two components capture the maximum amount of information (i.e. variance) in the original variables. From the output of our prepped recipe `pca_estimates`, we can examine how much variance each component accounts for:



In [ ]:
# Examine how much variance each PC accounts for
pca_estimates %>% 
  tidy(id = "pca", type = "variance") %>% 
  filter(str_detect(terms, "percent"))


theme_set(theme_light())
# Plot how much variance each PC accounts for
pca_estimates %>% 
  tidy(id = "pca", type = "variance") %>% 
  filter(terms == "percent variance") %>% 
  ggplot(mapping = aes(x = component, y = value)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("% of total variance")


This output tibbles and plots shows how well each principal component is explaining the original six variables. For example, the first principal component (PC1) explains about `84.93%` of the variance of the three variables. The second principal component explains an additional `7.63%`, giving a cumulative percent variance of `92.56%`. This is certainly better. It means that the first two variables seem to have some power in summarizing the original three variables.

Now that we have the data points translated to two dimensions PC1 and PC2, we can visualize them in a plot:


In [ ]:
# Visualize PC scores
features_2d %>% 
  ggplot(mapping = aes(x = PC1, y = PC2)) +
  geom_point(size = 2, color = "dodgerblue3")


### **Calculate WCSS for multiple cluster numbers to determine the right number of clusters**

Now here lies one of the fundamental problems with clustering - without known class labels, how do you know how many clusters to separate your data into?

One way we can try to find out is to use a data sample to create a series of clustering models with an incrementing number of clusters, and measure how tightly the data points are grouped within each cluster. A metric often used to measure this tightness is the *within cluster sum of squares* (WCSS), with lower values meaning that the data points are closer. You can then plot the WCSS for each model.

We'll use the built-in `kmeans()` function, which accepts a data frame with all numeric columns as it's primary argument to perform clustering. Let's also have the data on the same scale. We can use the recipes package to perform these transformations.


In [ ]:
# Data for clustering
clust_data <- recipe(~ ., data = data) %>% 
  step_normalize(all_predictors()) %>% 
  prep() %>% 
  bake(new_data = NULL)

# Print out data
clust_data %>% 
  slice_head(n = 5)


Now, let's explore the WCSS of different numbers of clusters.

We'll get to use `map()` from the [purrr](https://purrr.tidyverse.org/) package to apply functions to each element in list.

> [`map()`](https://purrr.tidyverse.org/reference/map.html) functions allow you to replace many for loops with code that is both more succinct and easier to read. The best place to learn about the [`map()`](https://purrr.tidyverse.org/reference/map.html) functions is the [iteration chapter](http://r4ds.had.co.nz/iteration.html) in R for data science.
>
> `broom::augment.kmeans()` accepts a model object and returns a tibble with exactly one row of model summaries. The summaries are typically goodness of fit measures, p-values for hypothesis tests on residuals, or model convergence information.


In [ ]:
set.seed(2056)
# Create 10 models with 1 to 10 clusters
kclusts <- tibble(k = 1:10) %>% 
  mutate(
    model = map(k, ~ kmeans(x = clust_data, centers = .x,
                            nstart = 20)),
    tot.withinss = map_dbl(model, ~ glance(.x)$tot.withinss))

# View results
kclusts

# Plot Total within-cluster sum of squares (tot.withinss)
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.5, color = "dodgerblue3") +
  geom_point(size = 2, color = "dodgerblue3")


We seek to **minimize** the the total within-cluster sum of squares, by performing K-means clustering. The plot shows a large reduction in WCSS (so greater *tightness*) as the number of clusters increases from one to two, and a further noticable reduction from two to three, then three to four clusters. After that, the reduction is less pronounced, resulting in an `elbow` 💪in the chart at around four clusters. This is a good indication that there are four reasonably well separated clusters of data points.

### **Use K-Means**

We can now go ahead and extract the clustering model where `k = 4`:

> `pull()`: used to extract a single column
>
> `pluck()`: used to index data structures such as lists


In [ ]:
# Extract the model based on 4 centroids
final_kmeans <- kclusts %>% 
  filter(k == 4) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


### **Plot the clustered points**

Great! Let's go ahead and visualize the clusters obtained. Care for some interactivity using `plotly`?


In [ ]:
# Load plotting libraries 
library(plotly)
library(paletteer)


# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, features_2d)  

# Plot cluster assignments
cluster_plot <- results %>% 
  ggplot(mapping = aes(x = PC1, y = PC2)) +
  geom_point(aes(shape = .cluster, color = .cluster), size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggsci::category10_d3")

# Make plot interactive
ggplotly(cluster_plot)


🤩🤩 Hopefully, the data has been separated into four distinct clusters.

### **Try agglomerative clustering**

Hierarchical clustering creates clusters by either a *divisive* method or *agglomerative* method. The `divisive` method is a `top down` approach starting with the entire dataset and then finding partitions in a stepwise manner. `Agglomerative clustering` is a `bottom up` approach in which each observation is initially considered as a single element cluster and at each step of the algorithm, two clusters that are most similar are combined into a bigger cluster until all points are members of one single big cluster.


In [ ]:
# For reproducibility
set.seed(2056)

# Distance between observations matrix
d <- dist(x = clust_data, method = "euclidean")

# Hierarchical clustering using Ward Linkage
hclust_ward <- hclust(d, method = "ward.D2")


The [factoextra](https://rpkgs.datanovia.com/factoextra/index.html) provides functions ([`fviz_dend()`](https://rdrr.io/pkg/factoextra/man/fviz_dend.html)) to visualize hierarchical clustering. Let's visualize the dendrogram representation of the clusters from the ward linkage method.



In [ ]:
library(factoextra)

# Visualize cluster separations
fviz_dend(hclust_ward, main = "Ward Linkage")


Although hierarchical clustering does not require one to pre-specify the number of clusters, one still needs to specify the number of clusters to extract.

We found out that the optimal number of clusters was 4. Let's color our dendrogram according to k = 4 and observe how observations will be grouped.


In [ ]:
# Visualize clustering structure for 3 groups
fviz_dend(hclust_ward, k = 4, main = "Ward Linkage")


We can now go ahead and `cut` the hierarchical clustering model into four clusters and extract the cluster labels for each observation associated with a given cut. This is done using `cutree()`



In [ ]:
# Hierarchical clustering using Ward Linkage
hclust_ward <- hclust(d, method = "ward.D2")

# Group data into 4 clusters
results_hclust <- tibble(
  cluster_id = cutree(hclust_ward, k = 4)) %>% 
  mutate(cluster_id = factor(cluster_id)) %>% 
  bind_cols(features_2d)

results_hclust %>% 
  slice_head(n = 5)


### **View the agglomerative cluster assignments**



In [ ]:
# Plot h-cluster assignmnet on the PC data
hclust_spc_plot <- results_hclust %>% 
  ggplot(mapping = aes(x = PC1, y = PC2)) +
  geom_point(aes(shape = cluster_id, color = cluster_id), size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggsci::category10_d3")

# Make plot interactive
ggplotly(hclust_spc_plot)
